# Klassifikation mit einem Neuronalen Netz

## Trainingsdaten

In [64]:
# %pip install torch
# %pip install torchvision
# %pip install matplotlib

In [65]:
# Import necessary libraries/modules
from torchvision import datasets, transforms  # Import datasets and transforms from the torchvision library
from torch.utils.data import TensorDataset, DataLoader  # Import TensorDataset and DataLoader from torch.utils.data
import torch  # Import the main PyTorch library
import torch.nn as nn  # Import the neural network module from PyTorch
import torch.nn.functional as F  # Import functional components of neural networks from PyTorch
import torch.optim as optim  # Import optimization algorithms from PyTorch
import matplotlib.pyplot as plt  # Import the matplotlib library for plotting
import numpy as np  # Import the NumPy library for numerical operations

# Define a function 'load' that loads data from a given file
def load(data):
    return np.load(data)['arr_0']  # Load data from the specified file using NumPy and return it

# Load the training and testing data and labels
train_data = load('Data\K49-data\k49-train-imgs.npz')  # Load training image data from a file
test_data = load('Data\K49-data\k49-test-imgs.npz')    # Load testing image data from a file

train_labels = load('Data\K49-data\k49-train-labels.npz')  # Load training labels from a file
test_labels = load('Data\K49-data\k49-test-labels.npz')    # Load testing labels from a file


## Definition des Neuronalen Netzes

In [66]:
# Set parameters for the feedforward neural network
input_size = 28*28  # Input size is 784 (assuming image size is 28x28 pixels)
hidden_size = 3999  # Number of neurons in the hidden layer
num_classes = 49    # Number of output classes (categories)
num_epochs = 15     # Number of training epochs
batch_size = 2048   # Batch size for data processing
learning_rate = 0.001  # Learning rate for the optimizer

# Convert the training data to the appropriate data type (float32)
train_data = train_data.astype(np.float32)

# Combine the training data and labels into a PyTorch TensorDataset
dataset = TensorDataset(torch.tensor(train_data), torch.tensor(train_labels))

# Create data loaders for training and testing data
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Define the architecture of the neural network
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        # Define the first hidden layer
        self.l1 = nn.Linear(input_size, hidden_size).to(dtype=torch.float32)  # Fully connected layer
        self.relu = nn.ReLU()  # Rectified Linear Unit (ReLU) activation function
        # Define the output layer
        self.l2 = nn.Linear(hidden_size, num_classes).to(dtype=torch.float32)  # Fully connected layer

    def forward(self, x):  # Forward pass through the network
        out = self.l1(x)  # Pass input through the first hidden layer
        out = self.relu(out)  # Apply ReLU activation
        out = self.l2(out)  # Pass through the output layer to get predictions
        return out

# Create an instance of the neural network model
model = NeuralNet(input_size, hidden_size, num_classes)


Wie erwartet sind die Gewichte des Neuronalen Netzes vor dem Training zufällig gewählt, so dass die Klassifikation misslingt.

## Training des Neuronalen Netz

In [67]:
# Define the loss function (CrossEntropyLoss) and the optimizer (Adam)
criterion = nn.CrossEntropyLoss()  # Loss function for classification problems
optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Adam optimizer with specified learning rate

# Train the model
total_step = len(train_loader)  # Total number of batches in the training data
for epoch in range(num_epochs):  # Loop through each training epoch
    for i, (train_data, train_labels) in enumerate(train_loader):  # Loop through each batch of data
        # Reshape the input images to have the shape (batch_size, input_size)
        train_data = train_data.reshape(-1, 28*28)

        # Forward pass: compute predicted outputs by passing inputs to the model
        outputs = model(train_data)

        # Calculate the loss using the defined criterion (CrossEntropyLoss)
        loss = criterion(outputs, train_labels)

        # Backward pass and optimization
        optimizer.zero_grad()  # Clear gradients from previous steps
        loss.backward()  # Perform backpropagation to compute gradients
        optimizer.step()  # Update model parameters using the computed gradients

        # Print training statistics
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/15], Step [100/114], Loss: 0.9813
Epoch [2/15], Step [100/114], Loss: 0.5547
Epoch [3/15], Step [100/114], Loss: 0.5478
Epoch [4/15], Step [100/114], Loss: 0.3465
Epoch [5/15], Step [100/114], Loss: 0.3423
Epoch [6/15], Step [100/114], Loss: 0.2657
Epoch [7/15], Step [100/114], Loss: 0.2063
Epoch [8/15], Step [100/114], Loss: 0.2056
Epoch [9/15], Step [100/114], Loss: 0.2087
Epoch [10/15], Step [100/114], Loss: 0.1596
Epoch [11/15], Step [100/114], Loss: 0.1645
Epoch [12/15], Step [100/114], Loss: 0.1442
Epoch [13/15], Step [100/114], Loss: 0.1179
Epoch [14/15], Step [100/114], Loss: 0.1720
Epoch [15/15], Step [100/114], Loss: 0.1665


## Test des Neuronalen Netzes

In [68]:
# In the test phase, we don't need to compute gradients for memory efficiency
with torch.no_grad():
    correct = 0  # Initialize the number of correctly predicted samples
    total = 0    # Initialize the total number of samples

    # Loop through the test data in batches
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)  # Reshape input images
        outputs = model(images)  # Forward pass to get model predictions
        _, predicted = torch.max(outputs.data, 1)  # Get the predicted class for each sample

        total += labels.size(0)  # Increment the total count by the batch size
        correct += (predicted == labels).sum().item()  # Count correct predictions in this batch

    # Calculate and print the test accuracy of the model
    print('Test Accuracy of the model on test images: {:.2f} %'.format(100 * correct / total))


Test Accuracy of the model on test images: 97.43 %


800min
hidden_size = 240100

num_classes = 49

num_epochs = 10

batch_size = 500

learning_rate = 0.001

Accuracy = 1.5587545456501624 %

47 min

hidden_size = 24010

num_classes = 49

num_epochs = 10

batch_size = 500

learning_rate = 0.001

Accuracy = 91.28913562713834 %

In [3]:
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

import torch

def predict_digit_pytorch(img, model):
    # Resize image to 28x28 pixels
    img = img.resize((28, 28))
    # Convert RGB to grayscale
    img = img.convert('L')
    img = np.array(img)
    # Reshaping and normalizing
    img = img.reshape(1, 28 * 28)
    img = img / 255.0

    # Convert to PyTorch tensor
    img_tensor = torch.tensor(img, dtype=torch.float32)

    # Predicting the class
    with torch.no_grad():
        outputs = model(img_tensor)
        _, predicted = torch.max(outputs, 1)

    return predicted.item()


class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.x = self.y = 0
        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg="white", cursor="cross")
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text="Recognise", command=self.classify_handwriting)
        self.button_clear = tk.Button(self, text="Clear", command=self.clear_all)
        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=tk.W)
        self.label.grid(row=0, column=1, pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")

    def classify_handwriting(self):
        HWND = self.canvas.winfo_id()  # Get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND)  # Get the coordinate of the canvas
        im = ImageGrab.grab(rect)
        digit = predict_digit_pytorch(im, model)
        self.label.configure(text=str(digit))


    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r = 8
        self.canvas.create_oval(self.x - r, self.y - r, self.x + r, self.y + r, fill='black')

app = App()
app.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "g:\Uni\Jupiter\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-3-6da93233e3d4>", line 44, in classify_handwriting
    digit, acc = predict_digit(im)
  File "<ipython-input-3-6da93233e3d4>", line 18, in predict_digit
    res = model.predict([img])[0]
NameError: name 'model' is not defined
